In [4]:
from collections import Counter
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 20)

In [5]:
import xgboost as xgb

In [6]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [7]:
from statistic import Statistic
from utils import UtilsKy
from analyzer import HelperAnalyzer, AnalyzerPrediction

In [8]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# kyw3
#path_data = '/mnt/files/workdata/work/merchants/merchant_33_kyw3_2020-06-05/04_experiments/ex_01_some_teach/'
db_teach = pd.read_csv(UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [5]:
# ky9
# path_data = '/mnt/files/workdata/work/merchants/merchant_32_ky9_2020-05-12_white_visa/04_experiments/'
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9, dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# for prod
# db_teach = pd.read_csv(UtilsKy.DB_TEACH_KY9_FOR_PROD, dtype=str)
# db_test = pd.read_csv(UtilsKy.DB_TEST_KY9_FOR_PROD , dtype=str)
# white = pd.read_csv(UtilsKy.WHITE_KY9 , dtype=str)

# db_teach = pd.read_csv(path_data + 'ex_05_ky9_xgb_jupiter_2020_07_08/db_teach_ky9_is_frequency_ip.csv', dtype=str)
# db_test = pd.read_csv(path_data + 'ex_05_ky9_xgb_jupiter_2020_07_08/db_test_ky9_is_frequency_ip.csv', dtype=str)

In [11]:
db_teach.columns

Index(['amount', 'amount_deviation', 'bank_currency', 'bin', 'city',
       'count_months_to_end_card', 'day_of_week', 'gender2', 'hour', 'id',
       'is_city_resolved', 'is_gender_undefined', 'latitude', 'longitude',
       'order_id', 'phone_2_norm', 'status'],
      dtype='object')

In [13]:
Statistic.get_table_value_counts(db_teach, 'status')

0    427164
1    6261  
Name: status, dtype: int64

In [14]:
Statistic.get_table_value_counts(db_test, 'status')

0    58107
1    755  
Name: status, dtype: int64

In [19]:
COL_FACTORS = ['bin', 'amount', 'bank_currency', 'hour', 'day_of_week', 'longitude', 'latitude', 'phone_2_norm'] 
# COL_FACTORS = COL_FACTORS + ['is_gender_undefined', 'is_city_resolved']

In [20]:
train = db_teach[COL_FACTORS]
test = db_test[COL_FACTORS]

In [21]:
train = train.apply(pd.to_numeric, errors="coerce")
test = test.apply(pd.to_numeric, errors="coerce")

In [22]:
replace_val = -9999
train.fillna(replace_val)
test.fillna(replace_val)
replace_val

-9999

In [23]:
train = train.values
test = test.values
label = db_teach.status

In [24]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [26]:
# result_df_amount = None
weight = analyzer_prediction.get_xgb_weight()

In [27]:
# Kyw3
max_depths =[3]
nrounds = [80, 90]
etas = [0.2, 0.35]

# Ky9
# max_depths =[5]
# nrounds = [70, 90]
# etas = [0.35, 0.4]

In [28]:
for nround in nrounds:
    for eta in etas:
        for max_depth in max_depths:
            config = {'max_depth': max_depth, 'learning_rate': eta
                      , 'n_estimators': nround
                     }
            model = xgb.XGBClassifier(**config)
            
            model.fit(train, label, eval_metric = 'auc', sample_weight=weight)
             
            test_pred = model.predict_proba(test)
            db_test["probability"] = test_pred[:, 1]

            description = '-' . join([str(elem) for elem in (max_depth, nround, eta)])                    
            result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")

In [29]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [30]:
stat_best.iloc[:,:11] 

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
4,3-90-0.2,4.87,12.21,15.98,18.08,20.47,22.28,23.41,26.09,40.83,138.53
2,3-80-0.35,6.79,14.39,16.88,18.45,19.77,20.04,20.70,24.39,36.88,137.55
0,3-80-0.2,3.86,11.67,16.29,17.64,19.49,22.12,23.00,27.23,41.17,137.35
6,3-90-0.35,4.85,11.88,15.12,16.34,17.24,19.14,19.14,24.91,38.34,128.68


In [93]:
# stat_best.iloc[:,:11] # Kyw3

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating
52,3-90-0.2-is_city_resolved-is_gender_undef,9.07,14.38,15.73,19.90,22.01,23.56,25.79,30.69,42.82,154.60
34,3-80-0.35-is_fr_net-is_gender_undef,7.20,17.02,18.59,19.83,20.96,21.31,23.38,26.68,42.36,152.64
38,3-90-0.35-is_fr_net-is_gender_undef,7.20,16.80,18.59,19.83,20.71,21.21,23.36,27.17,41.70,152.00
48,3-80-0.2-is_city_resolved-is_gender_undef,7.34,13.32,15.53,19.15,21.86,23.51,25.75,30.83,43.47,151.50
40,3-80-0.2-is_gender_undef,7.99,14.26,17.23,19.20,19.24,23.05,23.28,27.74,43.46,149.12
44,3-90-0.2-is_gender_undef,9.09,12.50,17.48,19.20,20.26,22.23,22.63,27.44,42.20,148.17
36,3-90-0.2-is_fr_net-is_gender_undef,7.71,12.42,15.79,18.83,21.75,24.04,24.70,29.90,41.34,147.74
32,3-80-0.2-is_fr_net-is_gender_undef,6.44,10.39,16.05,19.14,21.45,23.29,24.84,30.78,42.64,146.89
60,3-90-0.2-is_fr_net-is_city_resolved-is_gender_undef,5.64,10.66,16.80,18.97,20.86,22.31,25.15,30.71,40.79,144.43
56,3-80-0.2-is_fr_net-is_city_resolved-is_gender_undef,5.39,10.62,14.70,18.35,20.59,21.91,24.04,28.01,43.85,141.51
